In [1]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

SDK version: 1.19.0


In [13]:
from azureml.core import Workspace, Experiment

#ws = Workspace.get(name="udacity-project")
ws = Workspace.from_config()
ws.write_config(path='.azureml')
exp = Experiment(workspace=ws,name="udacity-project1")


print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

#run = exp.start_logging()


Workspace name: udacity-ml
Azure region: eastasia
Subscription id: 77aa4fe1-b94d-42b7-8b39-c4c15eb26acb
Resource group: res-classic-2


In [18]:
from azureml.core.compute import ComputeTarget, AmlCompute
import pandas as pd
# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

output = {}
output['Subscription ID'] = ws.subscription_id
output['Workspace'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Experiment Name'] = exp.name
pd.set_option('display.max_colwidth', -1)
outputDF = pd.DataFrame(data=output,index=[''])
outputDF.T

from azureml.core.compute_target import ComputeTargetException
### YOUR CODE HERE ###
cpu_cluster_name = 'hyper-ml'

#verify cluster does not already exsist
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print ('Found Exsisting Cluster')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
    max_nodes=2)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
compute_target.wait_for_completion(show_output=True)



Found Exsisting Cluster

Running


In [5]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

data = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path=data)


In [7]:
def clean_data(data):
    # Dict for cleaning data
    months = {"jan":1, "feb":2, "mar":3, "apr":4, "may":5, "jun":6, "jul":7, "aug":8, "sep":9, "oct":10, "nov":11, "dec":12}
    weekdays = {"mon":1, "tue":2, "wed":3, "thu":4, "fri":5, "sat":6, "sun":7}

    # Clean and one hot encode data
    x_df = data.to_pandas_dataframe().dropna()
    jobs = pd.get_dummies(x_df.job, prefix="job")
    x_df.drop("job", inplace=True, axis=1)
    x_df = x_df.join(jobs)
    x_df["marital"] = x_df.marital.apply(lambda s: 1 if s == "married" else 0)
    x_df["default"] = x_df.default.apply(lambda s: 1 if s == "yes" else 0)
    x_df["housing"] = x_df.housing.apply(lambda s: 1 if s == "yes" else 0)
    x_df["loan"] = x_df.loan.apply(lambda s: 1 if s == "yes" else 0)
    contact = pd.get_dummies(x_df.contact, prefix="contact")
    x_df.drop("contact", inplace=True, axis=1)
    x_df = x_df.join(contact)
    education = pd.get_dummies(x_df.education, prefix="education")
    x_df.drop("education", inplace=True, axis=1)
    x_df = x_df.join(education)
    x_df["month"] = x_df.month.map(months)
    x_df["day_of_week"] = x_df.day_of_week.map(weekdays)
    x_df["poutcome"] = x_df.poutcome.apply(lambda s: 1 if s == "success" else 0)

    y_df = x_df.pop("y").apply(lambda s: 1 if s == "yes" else 0)
    return x_df,y_df

import pandas as pd
# Using the same clean data function to clean the imported csv file dataset.
x, y = clean_data(ds)
print(type(x))
print(type(y))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>


In [8]:
y_dataframe=pd.DataFrame(y,columns=['y'])
y_dataframe.head(5)

,y
0,0
1,0
2,0
3,0
4,0


In [10]:
converted_data=pd.concat([x,y_dataframe],axis=1)

In [11]:
converted_data.to_csv('training/full_data.csv')

In [14]:
datastore=ws.get_default_datastore()
datastore

{
  "name": "workspaceblobstore",
  "container_name": "azureml-blobstore-96aa4fb1-b1db-4cdc-9207-5775fcc82f8a",
  "account_name": "udacityml3129747833",
  "protocol": "https",
  "endpoint": "core.windows.net"
}

In [15]:
datastore.upload(src_dir='training/',target_path='d/')


Uploading an estimated of 3 files
Uploading training\train.py
Uploaded training\train.py, 1 files out of an estimated total of 3
Uploading training\full_data.csv
Uploaded training\full_data.csv, 2 files out of an estimated total of 3
Uploading training\train_data.csv
Uploaded training\train_data.csv, 3 files out of an estimated total of 3
Uploaded 3 files


$AZUREML_DATAREFERENCE_49ad5cabd667449597165fdd044149b0

In [16]:
from azureml.core import Dataset
full_data=Dataset.Tabular.from_delimited_files(path=[(datastore,('d/full_data.csv'))])

In [19]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=full_data,
    label_column_name='y',
    n_cross_validations=5,
    compute_target=compute_target,
    iterations=100,
    max_concurrent_iterations=8)

In [20]:
automl_run=exp.submit(automl_config, show_output=True)

Running on remote.
No run_configuration provided, running on hyper-ml with default configuration
Running on remote compute: hyper-ml
Parent Run ID: AutoML_dfb58a58-23f4-4fbe-8c8e-7799c72fbab2

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+-------------------------------

        57   SparseNormalizer XGBoostClassifier             0:02:12       0.9099    0.9163
        60   SparseNormalizer XGBoostClassifier             0:01:18       0.8880    0.9163
        63   SparseNormalizer XGBoostClassifier             0:00:52          nan    0.9163
        66                                                  0:01:08          nan    0.9163
        67                                                  0:00:50          nan    0.9163
        61   SparseNormalizer XGBoostClassifier             0:01:20          nan    0.9163
        62   SparseNormalizer XGBoostClassifier             0:01:12       0.9097    0.9163
        64                                                  0:00:15          nan    0.9163
        65                                                  0:01:21          nan    0.9163
        68                                                  0:00:18          nan    0.9163
        69    VotingEnsemble                                0:02:10       0.9172    0.9172

In [21]:
automl_run.wait_for_completion(show_output=True)



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+--------------------------------------+
|Size of the smallest class       |Name/Label of the smallest class |Number of samples in the training data|
+=================================+=================================+======================================+
|3692                             |1                                |32950                                 |
+---------------------------------+----------

{'runId': 'AutoML_dfb58a58-23f4-4fbe-8c8e-7799c72fbab2',
 'target': 'hyper-ml',
 'status': 'Completed',
 'startTimeUtc': '2020-12-28T10:02:25.497281Z',
 'endTimeUtc': '2020-12-28T10:41:23.926422Z',
 'properties': {'num_iterations': '100',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'hyper-ml',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-project1","subscription_id":"77aa4fe1-b94d-42b7-8b39-c4c15eb26acb","resource_group":"res-classic-2","workspace_name":"udacity-ml","region":"eastasia","compute_target":"hyper-ml","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":100,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":5,"y_min":null,"y_max":null,"num_classes":null,"featurization":"auto","_ignore_packa

In [22]:
#Getting the best model output
best_run,model=automl_run.get_output()

In [23]:
#Exporting most optimal model obtained from AutoML using joblib
import joblib
joblib.dump(model,'optimal_model.joblib')

['optimal_model.joblib']

In [24]:
#Printing the best estimator for prediction from AutoML
print(model._final_estimator)

PreFittedSoftVotingClassifier(classification_labels=None,
                              estimators=[('44',
                                           Pipeline(memory=None,
                                                    steps=[('standardscalerwrapper',
                                                            <azureml.automl.runtime.shared.model_wrappers.StandardScalerWrapper object at 0x00000245B1B46EF0>),
                                                           ('xgboostclassifier',
                                                            XGBoostClassifier(base_score=0.5,
                                                                              booster='gbtree',
                                                                              colsample_bylevel=1,
                                                                              colsample_bynode=1,
                                                                              colsample_bytree...
                 

In [1]:
compute_target.delete()

NameError: name 'compute_target' is not defined